<a href="https://colab.research.google.com/github/ELehmann91/Thesis_Multilingual_Transferlearning/blob/master/Text_Annotation_Tool_BuBa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

# ECOICOP Annotation
  
Use this tool to annotate your product texts with the correspondant eCoiCop category. 


## Get code from GitHub
  
At first we need to download the code from Github to here.

In [1]:
%%capture
!pip install eli5
!git clone 'https://github.com/ELehmann91/Thesis_Multilingual_Transferlearning'
 
%cd Thesis_Multilingual_Transferlearning
import labeler_cc5
import coicop_model
import pandas as pd
import numpy as np
from tqdm import tqdm
import io

## Upload CSV / Excel from Local Drive

To upload from your local drive, start with the following code:

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Match_DE_Galeria_small.xlsx to Match_DE_Galeria_small.xlsx


It will prompt you to select a file. Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).

In [3]:
#df = pd.read_csv(io.BytesIO(uploaded['eantoclassifyECB.csv']),sep=',')
#df = pd.read_excel(io.BytesIO(uploaded['carrfour_trans_pred.xlsx']),encoding='ANSI')
df = pd.read_excel(uploaded['Match_DE_Galeria_small.xlsx'],encoding='unicode')
print(df.shape)
df[:2]

(1999, 7)


,PRODUKTART_BEZ,ABTEILUNG_BEZ,GESCHLECHT,WARENGRUPPE_BESCHREIBUNG1,category,url,label_cat5
0,DAMENWAESCHE,DA.-TAGW.,Damen,D-Slips,Tageswäsche-Damen,https://www.galeria.de/Schoeller-Damen-Hueftsl...,9999_Non-Food
1,OUTDOOR,OUTDOORBKL.,-,kurzarm He/Uni,Hemden - Bergsport / Wandern,NaN,NaN


## Upload from Google Drive

Loading CSV files stored in your google drive.

In [ ]:
from google.colab import drive, files
 
drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/Thesis_ecb_ecoicop'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


The commands will bring you to a Google Authentication step. You should see a screen with Google Drive File Stream wants to access your Google Account. After you allow permission, copy the given verification code and paste it in the box in Colab.
In the notebook, click on the charcoal > on the top left of the notebook and click on Files. Locate the data folder you created earlier and find your data. Right-click on your data and select Copy Path. Store this copied path into a variable and you are ready to go.

Specify the exact path to your file and the seperator of your file to read it in.

Print out the dataframe to see if the import is correct.

# Predict CoiCop Level 4
  
How does it work? - Word Embeddings (left)
* Words are translated in vectors which are learned representations
* Embeddings are a multidimensional space (often 300+ dimesnsion) and words which relate to each other are closer in this space
* Those spaces can be aligned for different languages
* The example shows a three dimensional space with embeddings for three different languages, the translation are idealy close neighbours and the distance between milk and cheese is closer than the distance betwenn milk and potato  
more information: https://towardsdatascience.com/introduction-to-word-embeddings-4cf857b12edc
  
How does it work? - Recurrent Neural Network (right)
* Now every word is represented by a vector and the input to the model is a sequence of vectors
* LSTM (Long Short Term Memory) Networks are good in solving sequential tasks, because they are able to remember information from previous states (words) and output a representation for the sentence 
* This representation is used to classify in to the coicop categories  
more information: https://towardsdatascience.com/a-beginners-guide-on-sentiment-analysis-with-rnn-9e100627c02e 




![embed](https://drive.google.com/uc?id=1AoleK5q47ZTkPCtvxDD6icEpcnLc9viE)

For the prediction you have to specify the names of certain columns in your dataframe which the tool will use.  
* name_col is the column with the product name (mandatory)
* category_col is the column with the category given by the supermarked (optional)
* url_col is the column with  the url to the product (optional)
* lang ist the language of the texts, supports 'de' and 'fr' (mandatory)
* label_cat5 is the column with labels,if labeld (optional)



In [4]:
# init predictor
  
name_col = 'WARENGRUPPE_BESCHREIBUNG1'
category_col = 'category'
url_col = 'url'
lang = 'de'
label_cat5 = None #'cc5' #'coicop4_str'

 #coicop_model #
CoiCop_Predictor = coicop_model.predictor(df
                                          , name_col
                                          , category_col
                                          , url_col
                                          , label_cat5
                                          , lang)


using name, category and words in url as input
using german embeddings
95% quantile no. of words per row is 11 (trained on 39)


In [5]:
df = CoiCop_Predictor.predict_proba()

100%|██████████| 19/19 [00:01<00:00, 14.46it/s]


In [10]:
df.iloc[111][:12]

PRODUKTART_BEZ                      STRUEMPFE                               
ABTEILUNG_BEZ                       STRUEMPFE                               
GESCHLECHT                                                                 -
WARENGRUPPE_BESCHREIBUNG1                                   Feinstrumpfhosen
category                                                   Neue WG verwenden
url                                                                      NaN
label_cat5                                                               NaN
url_text                                                                    
text                         Feinstrumpfhosen <sep> Neue WG verwenden <sep> 
cc3_pred                                                         99_Non-Food
cc4_pred                                                        999_Non-Food
cc5_pred                                                       9999_Non-Food
Name: 111, dtype: object

In [6]:
# Predicted ECOICOP Categories:
df['cc5_pred'].value_counts()

9999_Non-Food                                                            760
1211_Coffee                                                              144
1117_Breakfast cereals                                                   138
1192_Salt, spices and culinary herbs                                     128
1116_Pasta products and couscous                                         108
1154_Other edible oils                                                    96
1128_Other meat preparations                                              80
1125_Other meats                                                          62
2111_Spirits and liqueurs                                                 46
1193_Baby food                                                            46
1134_Frozen seafood                                                       41
1194_Ready-made meals                                                     31
1118_Other cereal products                                                31

In [8]:
if label_cat5 is not None:
    CoiCop_Predictor.test_performance()
    #CoiCop_Predictor.confusion_matrix()

## Explonation  
  
It shows the three best candidates for a product and highlights the words which contribute to the estimate (positiv: green, negative: red)
If you leave the the brackets empty it will show a random product. If you want the explonation for a particular product, you can enter the text.  
If you enter text it always needs to be in marks ''.

In [11]:
text = 'Feinstrumpfhosen <sep> Neue WG verwenden <sep>  '
categ= None #'1134_Frozen seafood'

CoiCop_Predictor.tell_me_why(text,categ)

Explanation(estimator="SGDClassifier(alpha=0.001, average=False, class_weight=None,\n              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,\n              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,\n              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',\n              power_t=0.5, random_state=RandomState(MT19937) at 0x7F7805DBC048,\n              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,\n              warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='9999_Non-Food', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[0] Feinstrumpfhosen', weight=1.235251966905317, std=None, value=1.0), FeatureWeight(feature='[5] sep', weight=0.5521560220846907, std=None, value=1.0), FeatureWeight(feature='[2] Neue', weight=0.4567805721618953, std=None, value=1.0), FeatureWeight(feature='[1] sep', weight=0.4382840742449622, std=None, value=1.0), FeatureWeight(feature='[4] verwenden', weight=0.22881065532395695, std=None, value=1.0), FeatureWeight(feature='[3] WG', weight=0.1838363791881941, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-4.92333466621069, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.1342689311090794, score=-1.8282149963016736, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='Feinstrumpfhosen <sep> Neue WG verwenden <sep>  ', spans=[('Feinstrumpfhosen', [(0, 16)], 1.235251966905317), ('sep', [(18, 21)], 0.4382840742449622), ('Neue', [(23, 27)], 0.4567805721618953), ('WG', [(28, 30)], 0.1838363791881941), ('verwenden', [(31, 40)], 0.22881065532395695), ('sep', [(42, 45)], 0.5521560220846907)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=3.095119669909016, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-4.92333466621069, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='1141_Fresh whole milk', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[5] sep', weight=0.3572171777647049, std=None, value=1.0), FeatureWeight(feature='[1] sep', weight=0.10469107811091977, std=None, value=1.0), FeatureWeight(feature='[0] Feinstrumpfhosen', weight=0.045213468434299846, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-2.5063673125919754, std=None, value=1.0), FeatureWeight(feature='[4] verwenden', weight=-0.5648256433877491, std=None, value=1.0), FeatureWeight(feature='[2] Neue', weight=-0.40923137761063483, std=None, value=1.0), FeatureWeight(feature='[3] WG', weight=-0.05416460968591055, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.04480475758372302, score=-3.027467218966345, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='Feinstrumpfhosen <sep> Neue WG verwenden <sep>  ', spans=[('Feinstrumpfhosen', [(0, 16)], 0.045213468434299846), ('sep', [(18, 21)], 0.10469107811091977), ('Neue', [(23, 27)], -0.40923137761063483), ('WG', [(28, 30)], -0.05416460968591055), ('verwenden', [(31, 40)], -0.5648256433877491), ('sep', [(42, 45)], 0.3572171777647049)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[], neg=[FeatureWeight(feature='<BIAS>', weight=-2.5063673125919754, std=None, value=1.0), FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=-0.52109990637437, std=None, value=None)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='1143_Preserved milk', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[0] Feinstrumpfhosen', weight=0.79343753690751, std=None, value=1.0), FeatureWeight(feature='[5] sep', weight=0.1758206537883063, std=None, value=1.0), FeatureWeight(feature='[4] verwenden', weight=0.17284866093774054, std=None, value=1.0), FeatureWeight(feature='[1] sep', weight=0.12293

# Annotation

For the annotation tool you have to specify the naes of certain columns in your dataframe which the tool will use.  
* Labeled_by takes your name and stores it after every item you labeld.  
* Text column 1 is the name of the column for the first line of text which will be displayed to help you labeling the product, usually this is the product name.  
* Text column 2 is the name of the column for the second line of text which will be displayed to help you labeling the product, this could be the category or the translation.  
* The URL column is the name of the column where the url of the scraped product is located (if it is in the dataframe). The url might be helpfull if you are unsure about the product nd want to look up the whole page on the website. If you do not have the url in your data you can leave it empty.  
* CoiCop 5 prediction is the column where the prediction is stored. The prediction will appear preselected in the dropdown menu.   
  
The data are sorted in a way that catogies with few labels come first, if predictions of those label are available.


In [12]:
 add_categories = ['12111 Hairdressing for men and children',
                   '12112 Hairdressing for women',
                   '12113 Personal grooming treatments',
                   '12121 Electric appliances for personal care',
                   '12131 Non-electrical appliances',
                   '12132 Articles for personal hygiene (wellness, esoteric beauty) ',
                   '06110 Pharmaceutical products',
                   '06131 Corrective eye-glasses and contact lenses',
                   '06110 Pharmaceutical products',
                   '06139 Other therapeutic appliances and equipment' ,
                   '5111_Household furniture',
                    '5112_Garden furniture',
                    '5113_Lighting equipment',
                    '5119_Other furniture and furnishings',
                    '5121_Carpets and rugs',
                    '5122_Other floor coverings',
                    '5123_Services of laying of fitted carpets and floor coverings',
                    '5130_Repair of furniture, furnishings and floor coverings',
                    '5201_Furnishing fabrics and curtains',
                    '5202_Bed linen',
                    '5203_Table linen and bathroom linen',
                    '5204_Repair of household textiles',
                    '5209_Other household textiles',
                    '5311_Refrigerators, freezers and fridge-freezers',
                    '5312_Clothes washing machines, clothes drying machines and dish washing machines',
                    '5313_Cookers',
                    '5314_Heaters, air conditioners',
                    '5315_Cleaning equipment',
                    '5319_Other major household appliances',
                    '5321_Food processing appliances',
                    '5322_Coffee machines, tea makers and similar appliances',
                    '5323_Irons',
                    '5324_Toasters and grills',
                    '5329_Other small electric household appliances',
                    '5330_Repair of household appliances',
                    '5401_Glassware, crystal-ware, ceramic ware and chinaware',
                    '5402_Cutlery, flatware and silverware',
                    '5403_Non-electric kitchen utensils and articles',
                    '5404_Repair of glassware, tableware and household utensils',
                    '5511_Motorised major tools and equipment',
                    '5512_Repair, leasing and rental of major tools and equipment',
                    '5521_Non-motorised small tools',
                    '5522_Miscellaneous small tool accessories',
                    '5523_Repair of non-motorised small tools and miscellaneous accessories',
                    '5611_Cleaning and maintenance products',
                    '5612_Other non-durable small household articles',
                    '5621_Domestic services by paid staff',
                    '5622_Cleaning services',
                    '5623_Hire of furniture and furnishings',
                    '5629_Other domestic services and household services'
                   ]

In [13]:
# init labeler

labeled_by = 'Erik'
text_column_1 = 'WARENGRUPPE_BESCHREIBUNG1'
text_column_2 = 'category'
url_column = 'url'
CoiCop_5_pred_col = 'cc5_pred'
Use_probabilities = False

#labeler_cc5
CoiCop_Labeler = labeler_cc5.labeler( labeled_by
                        , df
                        , text_column_1
                        , text_column_2
                        , url_column
                        , CoiCop_5_pred_col
                        , Use_probabilities
                        , add_categories)

Now you are ready to label, after executing the next line you will see:
* Select category you want to label
* Text 1
* Text 2
* URL
* Dropdown with one category which is the prediction for the product. If you are dissatisfied with it, you can click on the dropdown and select the right one.
* Next-Button will skip this product
* Save-Button will annotate the selected category to your dataframe
  
If you finished labeling just jump to the next line.

In [14]:
CoiCop_Labeler.start_to_label()

The next line will print out your current progress of annotating the data.

In [ ]:
CoiCop_Labeler.get_stats()

new labels: 0
in total 8473 of 9584 labeled ( 88.0 %)


Output the data including annotation.

In [ ]:
df= CoiCop_Labeler.output_labels()
df[:1]

,category,url,product_name,product_description,product_id_store,product_id,volume,qty,unit,price,scrape_time,shop,reduction,product,Coicop5_Final,Coicop4_Final,Coicop3_Final,Coicop2_Final,Predicted_COICOP_5_Category,Predicted_COICOP_4_Category,Predicted_COICOP_3_Category,Predicted_COICOP_2_Category,cc5,url_text,text,cc3_pred,cc4_pred,cc5_pred,1111_Rice,1112_Flours and other cereals,1113_Bread,1114_Other bakery products,1115_Pizza and quiche,1116_Pasta products and couscous,1117_Breakfast cereals,1118_Other cereal products,1121_Beef and veal,1122_Pork,1123_Lamb and goat,1124_Poultry,...,1164_Preserved fruit and fruit-based products,1171_Fresh or chilled vegetables other than potatoes and other tubers,1172_Frozen vegetables other than potatoes and other tubers,"1173_Dried vegetables, other preserved or processed vegetables",1174_Potatoes,1175_Crisps,1176_Other tubers and products of tuber vegetables,1181_Sugar,"1182_Jams, marmalades and honey",1183_Chocolate,1184_Confectionery products,1185_Edible ices and ice cream,1186_Artificial sugar substitutes,"1191_Sauces, condiments","1192_Salt, spices and culinary herbs",1193_Baby food,1194_Ready-made meals,1199_Other food products n.e.c.,1211_Coffee,1212_Tea,1213_Cocoa and powdered chocolate,1221_Mineral or spring waters,1222_Soft drinks,1223_Fruit and vegetable juices,2111_Spirits and liqueurs,2112_Alcoholic soft drinks,2121_Wine from grapes,2122_Wine from other fruits,2123_Fortified wines,2124_Wine-based drinks,2131_Lager beer,2132_Other alcoholic beer,2133_Low and non-alcoholic beer,2134_Beer-based drinks,2201_Cigarettes,2202_Cigars,2203_Other tobacco products,9999_Non-Food,max_score,labeled_by
711,"Frische & Kühlung Convenience Antipasti, Salat...",https://shop.rewe.de/p/star-gran-rag-con-fungh...,Star Gran Ragú con Funghi Porcini Tomatensauce...,Star Ragú con Funghi PorciniNudelsoße mit Flei...,p/star-gran-rag-con-funghi-porcini-tomatensauc...,68aec01c8c968c10022c797bfbfc93ff,"0,36 KG (1 KG = 13,11 €)","0,36",kilogram,4.72,2018-09-06 08:40:02,shop.rewe.de,False,https://shop.rewe.de/p/star-gran-rag-con-fungh...,1191,119,11,1,NaN,NaN,NaN,NaN,"1191_Sauces, condiments",star gran rag con funghi porcini tomatensauce ...,Star Gran Ragú con Funghi Porcini Tomatensauce...,11_Food,112_Meat,1121_Beef and veal,0.000784,0.005474,0.000501,0.004182,0.000294,0.003197,0.000771,0.005009,0.303897,0.003077,0.000217,0.002206,...,0.000171,0.000417,0.000347,0.010312,0.001596,0.00026,0.001808,0.002295,0.001729,0.001184,0.000354,0.000028,0.003317,0.009932,0.013511,0.00444,0.021621,0.016768,0.000805,0.001712,0.003842,0.001117,0.000988,0.00021,0.002624,0.00029,0.01161,0.000549,0.005678,0.001646,0.005133,0.010487,0.000398,0.003735,0.000213,0.00266,0.01582,0.030394,0.303897,None


## Save data to Local Drive

In [ ]:
from google.colab import files

df.to_csv('df_out.csv',sep='|')
files.download('df_out.csv')

## Save data to Google Drive

In [ ]:
df.to_csv(path+data_path+file_path,sep='|')
print(file_path,'saved',len(df),'observation',sum(df.cc5.isna()==False),'labeled')

training.csv saved 9584 observation 8473 labeled
